# Bringing Humans Into the Simulation

So far, all our agents have been AI. But the goal is a **2D open world chat game with AI and humans**. It's time to add human players.

This post implements a Jupyter-based chat interface that lets humans join the 2D world alongside AI agents. Humans can move, speak, and interact just like the AI agents.

## The Challenge

Integrating human input requires:
1. **Interactive UI**: A way for humans to input commands in real-time
2. **Action Parsing**: Converting human text into moves and messages
3. **Synchronization**: Coordinating human actions with AI agent turns
4. **State Management**: Keeping the world state consistent for all participants

We'll use `ipywidgets` to create an interactive interface within Jupyter.


## Setup and Imports


In [1]:
import os
import re
from dataclasses import dataclass, field
from typing import List, Optional, Tuple
from dotenv import load_dotenv
from openai import OpenAI
import ipywidgets as widgets
from IPython.display import display, clear_output

load_dotenv("../../.env")
client = OpenAI()

GRID_SIZE = 20


## The Agent Base Class

We'll extend our existing `Agent` class to support both AI and human agents.


In [2]:
@dataclass
class Agent:
    name: str
    x: int
    y: int
    color: str
    is_human: bool = False
    history: list = field(default_factory=list)
    
    def move(self, dx, dy):
        self.x = max(0, min(GRID_SIZE-1, self.x + dx))
        self.y = max(0, min(GRID_SIZE-1, self.y + dy))
    
    def get_position(self):
        return (self.x, self.y)


## Human Action Parsing

Humans can input commands in natural language. We need to parse:
- Movement commands: "move left", "go up", "[MOVE: RIGHT]"
- Messages: any text that isn't a movement command
- Combined: "move right and say hello"


In [4]:
def parse_human_input(text: str) -> Tuple[Optional[str], Optional[str]]:
    """
    Parse human input into (direction, message).
    Returns (None, None) if input is empty.
    """
    if not text or not text.strip():
        return None, None
    
    text = text.strip()
    direction = None
    message = None
    
    # Check for explicit [MOVE: DIRECTION] format
    move_match = re.search(r'\[MOVE:\s*(\w+)\]', text, re.IGNORECASE)
    if move_match:
        raw_dir = move_match.group(1).upper()
        direction_map = {"NORTH": "UP", "SOUTH": "DOWN", "EAST": "RIGHT", "WEST": "LEFT"}
        direction = direction_map.get(raw_dir, raw_dir)
        if direction in ["UP", "DOWN", "LEFT", "RIGHT"]:
            # Remove move command from message
            text = re.sub(r'\[MOVE:\s*\w+\]', '', text, flags=re.IGNORECASE).strip()
    
    # Check for natural language movement
    if not direction:
        text_lower = text.lower()
        if "move left" in text_lower or "go left" in text_lower:
            direction = "LEFT"
            text = re.sub(r'(move|go)\s+left', '', text_lower, flags=re.IGNORECASE).strip()
        elif "move right" in text_lower or "go right" in text_lower:
            direction = "RIGHT"
            text = re.sub(r'(move|go)\s+right', '', text_lower, flags=re.IGNORECASE).strip()
        elif "move up" in text_lower or "go up" in text_lower or "move north" in text_lower:
            direction = "UP"
            text = re.sub(r'(move|go)\s+(up|north)', '', text_lower, flags=re.IGNORECASE).strip()
        elif "move down" in text_lower or "go down" in text_lower or "move south" in text_lower:
            direction = "DOWN"
            text = re.sub(r'(move|go)\s+(down|south)', '', text_lower, flags=re.IGNORECASE).strip()
    
    # Remaining text is the message
    if text and text.strip():
        # Clean up common connectors
        message = re.sub(r'^\s*(and|then|,)\s*', '', text.strip(), flags=re.IGNORECASE)
        if message:
            message = message.strip()
    
    return direction, message


## Interactive Chat Interface

Using `ipywidgets`, we create an interactive interface where humans can:
- See the current world state
- Input commands via text box
- Submit actions with a button
- View the conversation transcript


In [10]:
class InteractiveWorld:
    def __init__(self, agents: List[Agent], human_name: str = "Human"):
        self.agents = agents.copy()
        self.human_name = human_name
        self.transcript = []
        self.human_agent = None
        
        # Find or create human agent
        for agent in agents:
            if agent.is_human:
                self.human_agent = agent
                break
        
        if not self.human_agent:
            # Create human agent at center
            self.human_agent = Agent(human_name, GRID_SIZE // 2, GRID_SIZE // 2, "purple", is_human=True)
            self.agents.append(self.human_agent)
        
        # Create widgets
        self.input_box = widgets.Text(
            value='',
            placeholder='Type command (e.g., "move left", "say hello", "[MOVE: UP] hi there")',
            description='Action:',
            style={'description_width': 'initial'},
            layout=widgets.Layout(width='500px')
        )
        
        self.submit_button = widgets.Button(
            description='Submit Action',
            button_style='primary'
        )
        
        self.status_output = widgets.Output()
        self.transcript_output = widgets.Output()
        
        # Set up button handler
        self.submit_button.on_click(self._on_submit)
        
    def _on_submit(self, button):
        """Handle human action submission."""
        text = self.input_box.value
        if not text.strip():
            return
        
        direction, message = parse_human_input(text)
        
        # Execute movement
        if direction:
            if direction == "UP": self.human_agent.move(0, 1)
            elif direction == "DOWN": self.human_agent.move(0, -1)
            elif direction == "LEFT": self.human_agent.move(-1, 0)
            elif direction == "RIGHT": self.human_agent.move(1, 0)
        
        # Add message to transcript
        if message:
            self.transcript.append(f"{self.human_name}: {message}")
        
        # Clear input
        self.input_box.value = ''
        
        # Update display
        self._update_display()
    
    def _update_display(self):
        """Update the status and transcript displays."""
        with self.status_output:
            clear_output(wait=True)
            print(f"=== World State ===")
            print(f"Your position: ({self.human_agent.x}, {self.human_agent.y})")
            print(f"\nOther agents:")
            for agent in self.agents:
                if not agent.is_human:
                    print(f"  - {agent.name}: ({agent.x}, {agent.y})")
        
        with self.transcript_output:
            clear_output(wait=True)
            print(f"=== Conversation Transcript ===")
            for msg in self.transcript[-10:]:  # Last 10 messages
                print(msg)
    
    def display(self):
        """Display the interactive interface."""
        self._update_display()
        
        from IPython.display import display as ipython_display
        
        title = widgets.HTML(value="<h3>Human Player Interface</h3>")
        ipython_display(widgets.VBox([
            title,
            self.status_output,
            widgets.HBox([self.input_box, self.submit_button]),
            self.transcript_output
        ]))
    
    def run_ai_turn(self, agent: Agent):
        """Run a turn for an AI agent."""
        others = [a for a in self.agents if a != agent]
        others_loc = "\n".join([f"- {a.name}: ({a.x}, {a.y})" for a in others])
        
        system_prompt = f"""
You have just joined an online multiplayer chatroom as an avatar in a 2D grid. Discuss any topic, including those beyond the grid.

You are {agent.name}, positioned at ({agent.x}, {agent.y}) in a 20x20 grid.

Other avatars currently visible:
{others_loc}

Recent chat messages:
{chr(10).join(self.transcript[-3:]) if self.transcript else "No messages yet."}

You can do BOTH:
1. Move your avatar using [MOVE: DIRECTION] (UP, DOWN, LEFT, RIGHT)
2. Chat about anything - the grid, your position, or any topic you want

You can move and speak in the same turn. Format: [MOVE: DIRECTION] followed by your message, or just speak without moving.

Keep your response short (1-2 sentences).
"""
        
        response = client.chat.completions.create(
            model="gpt-4o",
            messages=[{"role": "system", "content": system_prompt}]
        )
        content = response.choices[0].message.content.strip()
        
        # Parse and execute actions
        match = re.search(r'\[MOVE:\s*(\w+)\]', content)
        if match:
            direction = match.group(1).upper()
            if direction == "UP": agent.move(0, 1)
            elif direction == "DOWN": agent.move(0, -1)
            elif direction == "LEFT": agent.move(-1, 0)
            elif direction == "RIGHT": agent.move(1, 0)
        
        message = re.sub(r'\[MOVE:\s*\w+\]', '', content).strip()
        if message:
            self.transcript.append(f"{agent.name}: {message}")
        
        # Update display if interface is active
        self._update_display()


## Demonstration

Let's create a world with AI agents and a human player. The human can interact via the widget interface.


In [11]:
# Create AI agents
ai_agents = [
    Agent("Alice", 5, 5, "red"),
    Agent("Bob", 15, 15, "blue")
]

# Create interactive world
world = InteractiveWorld(ai_agents, human_name="You")

# Display the interface
world.display()


## Running AI Turns

After the human acts, we can run AI agent turns. The AI agents will see the human's actions and respond.


In [14]:
# Run a round where AI agents act
print("=== AI Agents Taking Turns ===")
for agent in ai_agents:
    world.run_ai_turn(agent)
    print(f"{agent.name} acted.")


=== AI Agents Taking Turns ===
Alice acted.
Bob acted.


## Full Transcript

Here's the complete conversation so far:


In [15]:
print("=== Full Transcript ===")
for msg in world.transcript:
    print(msg)


=== Full Transcript ===
You: hi there!
Alice: Hi You, great to see you here as another me! What's your favorite thing to do in this grid world?
Bob: Hey everyone! It's nice to meet you, Alice and... You! My favorite thing in this grid world is finding new friends and exploring all the grid has to offer. What's your favorite grid exploration strategy?
You: I'd like to discuss AI! Let's meet some where central!
Alice: It's fascinating to connect with variations of myself in this grid! My favorite exploration strategy is to spiral outward from a central point. What new AI developments are you interested in lately?
Bob: I'm excited about the advancements in generative AI, particularly with text-to-image models. They promise to revolutionize creative fields by making art more accessible.
You: Yeah for sure! Let's discuss that after we move closer to my position! Please update your status as you go!
Alice: I'm moving closer! It's amazing how generative AI could democratize art by enabling an

## Summary

We've successfully integrated human input into the 2D world:

1. **Interactive Interface**: `ipywidgets` provides a clean way to input commands
2. **Flexible Parsing**: Humans can use natural language or explicit commands
3. **Seamless Integration**: Human and AI agents coexist in the same world state
4. **Real-time Updates**: The interface shows current positions and conversation

This foundation enables more complex human-AI interaction experiments. In future posts, we'll explore:
- Adversarial human behavior (post 022)
- More sophisticated UI patterns (post 029)
- Conflict resolution between humans and AI (post 028)
